## I. Experiments

**!!! all mels were made by the initial MelSpec class (with shapes problems)**

### A. Supreme run

Of course, when you deal with such a consuming task in terms of computation and time, there is no time for various full-scope experiments. Hence, I've chosen a specific strategy of small launches, where the configurations (losses, small features, learning rates, etc.) are ran for approximately an hour in order to select the most resultative one. Of course, such an approach has a key issue that no fundamental difference could be seen on first 5-10k operations. Nevertheless, at lease it becomes possible to reduce certainly bad ideas. So, in this particular paragraph we will talk about the major experiment (>10 hours), while the following ones will be devoted to various alterations, taken place at the beginning.

**Configuration**: 
> full implementation of V1
>
> article's training hyperparameters; weight norm only for mpd
>
> $\mathcal{L}_G = \mathcal{L}_{Mel} * 45 + \mathcal{L}_{feature} * 2 + \mathcal{L}_{Gan}$

**Description** 

Yes, basically what we have for the experiment is a full reimplementation of the paper (except the contradicting moment with G normalization). While most of the minor changes I tried didn't affect the training manner in any sense, the decision was to simply follow the text we have without any additional tricks. Basically, what I've got stronly reminds me of my experiment with pix2pix model in image generation task. We have discriminator falling down at some point from high loss rates to low, after approaching it's asymptote at some value in betweem the peak and fall, where it resiles till the end of the training. This is quite logical: the real & fake waves are easy to distinguish because of a low quality of generator's output. On the other hand, generator initially grows up in terms of losses since D becomes rapidly better in identification. However, then D reaches the bottom and generator starts to producing more realistic sounds, making D to grow in losses and reach some asymptote as a maximum of capabilities.

Well, here we approach the key issue. While I scrutinized each page of the paper & carefully transferred it to the code, the generator's output are horrific for no reason. After checking dozens of times each aspect of dataset & model, the only answer I have is that the model didn't train enough. Nevertheless, during this experiment I've turned off the mel loss at some point, playing the biggest role in total loss of generator. However, while still some positive trend can be seen, no pure human voice is heard no matter what I tried.

Attempts to fix:

> Reinit the optimizers thinking that the problem is with the decreased learning rate, making alterations to weights negligible
>
> Remove the melloss, being the greatest value in the loss of generator sum, thinkg that it will make the algorithm pay more attention: $\mathcal{L}_G = \mathcal{L}_{feature} * 2 + \mathcal{L}_{Gan}$
>
> Turn off the trainer discriminator, so we only work with generator
>
> Reinit the discriminator, so it doesn't become too good in identifying fake audio

During all such operations several minor errors were made (e.g. weight decay value was given to schedulers' gamma instead of optimizers or no normalization for the first subscaleD). So, after the errors were found, I've decided to run another attempt with identical hyperparameters (initially, I wanted to continue the training, but understood that trained for so long model woln't be able to retrain).

**Graphs**
    
<center><img src="./img/D_supreme.png" width="700" height="400" border="1">

*Graph A.1 Result for entire supreme experiment's discirminator total loss*

<img src="./img/G_supreme.png" width="700" height="400" border="1">

*Graph A.2 Result for entire supreme experiment's generator total loss*</center>


### B. (No) weight-norm run

**Configuration**: 
> full implementation of V1
>
> article's training hyperparameters; weight norm 1) for everything 2) for nothing
>
> $\mathcal{L}_G = \mathcal{L}_{Mel} * 45 + \mathcal{L}_{feature} * 2 + \mathcal{L}_{Gan}$

**Description** 

One of the experiments I made in order to find the best solution. Here we have two possible versions: 1) no weight normalization at all 2) weight normalization everywhere including generator. Be quick to say the greatest effect we faced were incredibly high values for losses at the first spets of training. However, after we reached approximately same results as in the supreme run. One thing to mention is that normalization for generator has improved the performance of D. Nevertheless, as a final decision I have made the baseline supreme run in full accordance with paper. This seems rational as I have no understanding of the outcome on the large-scale. 

**Graphs**

<center><img src="./img/D_norm.png" width="700" height="400" border="1">

*Graph B.1 Results for discirminators' total losses during experiments with/without generator's weight normalization*

<img src="./img/G_norm.png" width="700" height="400" border="1">

*Graph B.2 Results for generators' total losses during experiments with/without generator's weight normalization*</center>


### C. Version.2

**Configuration**: 
> full implementation of V2
>
> article's training hyperparameters; weight norm for D
>
> $\mathcal{L}_G = \mathcal{L}_{Mel} * 45 + \mathcal{L}_{feature} * 2 + \mathcal{L}_{Gan}$

**Description** 

One of the experiments I made in order to find the best solution. Here I simply wanted to move from version one, requiring more calculations because of the initial upconvolution (differs in versions by four times). However, I was understanding all the possible outcomes, both cons, such as worse generalization capability, and pros, such as higher value for steps per sec. After an hour-and-a-half run I've seen merely the same quality with higher speed. This might be due to an error I've made, which forced all the models to achieve constant generator loss. However, in my supreme run I decided to stay with version 1 after diving quite more into the paper's point and messages in channel of DLA course, having no resources for launching both versions one by one on a large scale. At the same time, it still seems to me that for our task requiring not astonishing quality we could run on version 2.

### D. The winner

**Configuration**: 
> full implementation of V1
>
> article's training hyperparameters; weight norm for D
>
> $\mathcal{L}_G = \mathcal{L}_{Mel} * 45 + \mathcal{L}_{feature} * 2 + \mathcal{L}_{Gan}$


**Description** 

As you can see, the configuration provided for this particular experiment is identical to the one of Supreme. Then, why would I underscore any specific run as a new experiment? Well, the answer is quite simple: because the results have drastically altered. It was caused by solving a really tiny mistake, which cost me hours of training and other hours of searching. It basically lied in the model's optimization process, the mel loss calculation particularly. What I was doing is calculating mel for fake wave & comparing it to the calculated mel for real wave. However, for some reason such an approach was misplacing the spectra or resulting in someo other problem. So, what I should have basically done is to compare mel of fake wave to the input correct mel. When I've found out the issue, the sun rose (it can be seen on the graph, where mel loss "suddenly" falls). 

Now we have a far better situation. What happens is that mel loss has started giving good gradients to the generator, which in turn gained an opportunity to produce better waves (paying attention to what their mels should look like). The final outcome can be found on the discriminator's graph, where the loss slowly grows up since G produces with every step better and better audio.

The graphs presented below do not depict the entire story because second part of steps was made closer to the deadline. Nevertheless, the prime trend doesn't alter throught out all the steos, which can be proved by [W&B logs](https://wandb.ai/aborevsky/Neural%20Vocoder/runs/1frvzg26).

<p align="center" width="100%">
    <img width="32%" src="./img/feat_winner.png" border="1"> 
    <img width="32%" src="./img/gen_winner.png" border="1"> 
    <img width="32%" src="./img/mel_winner.png" border="1"> 
</p>

*Graph D.1 Results for TheWinner experiment a) feature loss b) generator loss c) mel loss*

<p align="center" width="100%">
    <img width="49%" src="./img/G_winner.png" border="1"> 
    <img width="49%" src="./img/D_winner.png" border="1"> 
</p>

*Graph D.2 Results for TheWinner experiment a) total G loss b) total D loss*

## II. Pipeline

[Generator state dict](https://www.dropbox.com/s/jlhhb19ja9n613g/g_winner?dl=0)

[Trainer checkpoint](https://www.dropbox.com/s/c9r8g8y06s7tilp/checkpoint-epoch70.pth?dl=0)

### Quick inference

In [8]:
#!g1.1
from configs import *
from datasets import *
from trainer import *
from submodules import *
import torch
import numpy
from torch.utils.data import DataLoader

train_config = TrainConfig()
val_dataset = EvalDataset(train_config)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0
)

In [6]:
#!g1.1
G_winner = Generator(train_config)
G_winner.load_state_dict(torch.load('g_winner'))

<All keys matched successfully>

In [ ]:
#!g1.1
with torch.no_grad():
    for i, batch in enumerate(val_loader):
        eval_out = G_winner(batch['mel'])
        eval_out = eval_out.detach().cpu()
        torchaudio.save("val_{0}.wav".format(i), eval_out[0], 22050)

### Training

In [21]:
#!g1.1
!python3 train.py


## III. Evaluation

### Features' workability assessment

1. Weight Norms

So, this is quite an intresting point. For real, a lot of attention in both explored papers [MelGAN](https://arxiv.org/pdf/1910.06711.pdf) & [HifiGAN](https://arxiv.org/pdf/2010.05646.pdf) is paid to the positive role of weight normalization for discriminator and generator. While I did not found a vast explanation of the reasons for such a trend, all experiments they provide tend to show their truth. Nevertheless, on the first steps of this work I've really missed this particular idea, so my first experiments were ran without any normalization except the one I took from [pix2pix](https://arxiv.org/abs/1611.07004) implementation. However, no outstanding alterations were found between the two launches. Nevertheless, the authors of MelGAN mention that the major idea lies near the resulting audio's quality (removal of extra noise as an outcome of weight normalization). So, probably my assertion of identical results for with/without WN architectures is quite compromised: the former was trained for longer period. Final word to say is that, of course, I decided not to pretend for being a genius and thus simply relied on authors' proposal. That means that only a HifiGAN's idea of discriminator normalization was introduced, while ignoring the MelGan approach to normalize everything.

2. V1 vs V2

Thanks to authors, they have provided us with already consistent configurations for their model in almost each aspect (it would be also great to get quite more descriptions of it, than two paragraphs). While three variants are profoundly described, two of them are majorly identical except the introducing upsampling (from input channels to $h_u$). So, let's call the first one "big" ($h_u$ = 512) & another - "small" ($h_u$=128). Having low scope runs for both approaches, I didn't found sufficient alterations (except the little for steps per second). The losses still appeared to be identical on the first stages. Of course, it is not enough to state the zero influence of this hyperparameter. Sure, if having more time, I'd be happy to run both configurations for sufficient time. Nevertheless, the decision was to introduce the first version (time/epoch looked reasonably plus tha it a classic DL approach to make smth bigger for better quality!)). Still, there is not enough room to assert the workability of small version on large scale, while the initialization has processed adequately.

3. Losses

There were some minor experiments devoted to removing some loss functions from the training process (feature loss, mel loss). While they are quite reasonable for DLA sphere, I being mainly outside this sphere have faced both for the first time. The only word in favor for expanding standard GAN's set of losses were words of authors. Thus, I decided to calmly introduce loss functions, starting from mandatory ones (GANLoss & DLoss). Of course, the plots are incomparable with each other since the loss functions' values are deeply dependent on the number of summed up values. Nevertheless, the low-scale runs I reviewed have shown identical quality of audio on the entire training spectrum (around 5k steps). But, I fully understand how negligible is this number of steps compared to the required one. Therefore, since being asked to simply implement the original code, I fully relied on authors' concept (they assert *The
result is also worse than the baseline. The experiment on the mel-spectrogram loss shows that it
helps improve the quality, and we observed that the quality improves more stably when the loss is
applied.*). Hence, the final generator loss is fully identical to the one shown in paper. However, since authors ignore the padding issue (different sizes) during mel comparison, there might be issues with training model to put always zero (no voice) in the end.

---

### Challenges

1. Datasphere & memory

Well, while it is obviously not the challenge of the given task itself, still working environment was enough problematic side to be mentioned. Datasphere has shown itself in the worst mannet ever since we had began to work here (and I can assure you that there already were plenty of cases). Especially, issues were arising when I was moving to GPU: the environment was constantly loading and committing states, each taking up to 15 minutes, while the resource itself was constantly losing connections and so on. Of course, major challenges I faced at this view were memory issues since cuda capabilities were not enough to proceed the entire waves in batch equal to 16, following the authors' advice. The most intresting thing here is that the authors themselves were cropping dataset's waves, not adding a single word about that to the paper.

2. Divide & conquer

Looking through the paper once and once again, it's easy quite remarkable how simple is each block itself, both in generator & discriminator. However, when constructing the comprehensive architecture out of these blocks, I found out the substantial extent to which I was leaving a minor errors behind. It is a direct consequence of a high abstact level of all the objects being inextricably bounded with each other through dozens of skip-connections. Hence, I had to come back to the model itself pretty many times, while being already quite acquintant with the GAN concept. Also, you find yourself somewhere in the middle of various upsampling and convolutions, all being set by extensive hyperparameters' libraries. While everything seemed quite easy on the research stage (lectures, seminars, paper revision), the embodiement itself, as usually, appeared to be far more consuming both in time and energy.

3. Authors' schemes

Vast part of the entire implemention is devoted to overwriting from paper's plans of both discriminator and generator. While they were designed in a really nice manner, still it took sufficient time to firstly rewrite them in a more pleasant manner (at least for me), get the idea of what is reapeated, how many times and with what particular hyperparameters, so the code would be come a less challenging this to make. Moreover, some aspects of authors' idea were not depicted in text (as for waves sizes, normalization of sums), another were conflicting with the original MelGAN article, to which they refer. Also, absence of any loss graphs makes the entire process quite more desperate, since no understanding of how it should look like is present. Nevertheless, while minor experience with GAN model implementation have empowered me in some sence, numerous local hints being ignored in the article have sufficiently complicated my life. 

4. Time

Another deeply problematic side of the project is the required time for training. Not only the training process consumes horrific amount of time to start producing some 
reasonable quality, it also gives almost no opportunity for experiment on a long range. As it was already mentioned, few information about the perfect manner of this process from authors together with no free time for innumerable experiments forcened to act blindly. It means that after a series of short-play runs was launched (each continuing for approximately an hour), the best configuration was chosen and set to be the only truly tested on full scale. While it looked quite reasonably to check the authors' implementation (so, getting the baseline for our future experiments), there was neither time or permission. 